In [1]:
# ЗАДАНИЕ 1
import sqlite3

DB_PATH = "university.db"

def create_database(db_path: str = DB_PATH) -> None:
    try:
        with sqlite3.connect(db_path) as conn:
            conn.execute("PRAGMA foreign_keys = ON;")
            cur = conn.cursor()

            cur.execute("""
                CREATE TABLE IF NOT EXISTS students (
                    id INTEGER PRIMARY KEY AUTOINCREMENT,
                    first_name TEXT NOT NULL,
                    last_name TEXT NOT NULL,
                    group_name TEXT NOT NULL,
                    admission_year INTEGER NOT NULL,
                    average_grade REAL,
                    created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP
                );
            """)

            cur.execute("""
                CREATE TABLE IF NOT EXISTS courses (
                    id INTEGER PRIMARY KEY AUTOINCREMENT,
                    course_name TEXT UNIQUE NOT NULL,
                    instructor TEXT NOT NULL,
                    credits INTEGER NOT NULL,
                    created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP
                );
            """)

            conn.commit()
            print("OK: university.db и таблицы students/courses созданы.")
    except sqlite3.Error as e:
        print(f"Ошибка при создании базы данных: {e}")

create_database()

OK: university.db и таблицы students/courses созданы.


In [2]:
# ЗАДАНИЕ 2
import sqlite3
from typing import Optional, List, Dict, Any

DB_PATH = "university.db"

def _get_conn():
    conn = sqlite3.connect(DB_PATH)
    conn.execute("PRAGMA foreign_keys = ON;")
    conn.row_factory = sqlite3.Row
    return conn

def add_student(first_name: str, last_name: str, group_name: str, admission_year: int, average_grade: Optional[float]=None) -> Optional[int]:
    try:
        with _get_conn() as conn:
            cur = conn.cursor()
            cur.execute(
                "INSERT INTO students (first_name, last_name, group_name, admission_year, average_grade) VALUES (?, ?, ?, ?, ?);",
                (first_name, last_name, group_name, admission_year, average_grade)
            )
            return cur.lastrowid
    except sqlite3.Error as e:
        print(f"add_student error: {e}")
        return None

def get_all_students() -> List[Dict[str, Any]]:
    try:
        with _get_conn() as conn:
            cur = conn.cursor()
            cur.execute("SELECT * FROM students ORDER BY id;")
            return [dict(r) for r in cur.fetchall()]
    except sqlite3.Error as e:
        print(f"get_all_students error: {e}")
        return []

def get_students_by_group(group_name: str) -> List[Dict[str, Any]]:
    try:
        with _get_conn() as conn:
            cur = conn.cursor()
            cur.execute("SELECT * FROM students WHERE group_name = ? ORDER BY id;", (group_name,))
            return [dict(r) for r in cur.fetchall()]
    except sqlite3.Error as e:
        print(f"get_students_by_group error: {e}")
        return []

def update_student_grade(student_id: int, new_grade: float) -> bool:
    try:
        with _get_conn() as conn:
            cur = conn.cursor()
            cur.execute("UPDATE students SET average_grade = ? WHERE id = ?;", (new_grade, student_id))
            return cur.rowcount > 0
    except sqlite3.Error as e:
        print(f"update_student_grade error: {e}")
        return False

def delete_student(student_id: int) -> bool:
    try:
        with _get_conn() as conn:
            cur = conn.cursor()
            cur.execute("DELETE FROM students WHERE id = ?;", (student_id,))
            return cur.rowcount > 0
    except sqlite3.Error as e:
        print(f"delete_student error: {e}")
        return False

sid = add_student("Иван", "Петров", "ГР-01", 2023, 4.5)
add_student("Мария", "Иванова", "ГР-01", 2023, 4.8)
add_student("Алексей", "Сидоров", "ГР-02", 2023, 3.9)
print("Все:", get_all_students())
print("ГР-01:", get_students_by_group("ГР-01"))
if sid:
    update_student_grade(sid, 4.9)
delete_student(2)
print("После обновления/удаления:", get_all_students())


Все: [{'id': 1, 'first_name': 'Иван', 'last_name': 'Петров', 'group_name': 'ГР-01', 'admission_year': 2023, 'average_grade': 4.5, 'created_at': '2026-01-18 17:00:15'}, {'id': 2, 'first_name': 'Мария', 'last_name': 'Иванова', 'group_name': 'ГР-01', 'admission_year': 2023, 'average_grade': 4.8, 'created_at': '2026-01-18 17:00:15'}, {'id': 3, 'first_name': 'Алексей', 'last_name': 'Сидоров', 'group_name': 'ГР-02', 'admission_year': 2023, 'average_grade': 3.9, 'created_at': '2026-01-18 17:00:15'}]
ГР-01: [{'id': 1, 'first_name': 'Иван', 'last_name': 'Петров', 'group_name': 'ГР-01', 'admission_year': 2023, 'average_grade': 4.5, 'created_at': '2026-01-18 17:00:15'}, {'id': 2, 'first_name': 'Мария', 'last_name': 'Иванова', 'group_name': 'ГР-01', 'admission_year': 2023, 'average_grade': 4.8, 'created_at': '2026-01-18 17:00:15'}]
После обновления/удаления: [{'id': 1, 'first_name': 'Иван', 'last_name': 'Петров', 'group_name': 'ГР-01', 'admission_year': 2023, 'average_grade': 4.9, 'created_at': '2

In [3]:
# ЗАДАНИЕ 3
import sqlite3
from typing import Optional, List, Dict, Any

DB_PATH = "university.db"

def _get_conn():
    conn = sqlite3.connect(DB_PATH)
    conn.execute("PRAGMA foreign_keys = ON;")
    conn.row_factory = sqlite3.Row
    return conn

def ensure_relations_tables() -> None:
    try:
        with _get_conn() as conn:
            cur = conn.cursor()
            cur.execute("""
                CREATE TABLE IF NOT EXISTS student_courses (
                    student_id INTEGER NOT NULL,
                    course_id INTEGER NOT NULL,
                    PRIMARY KEY (student_id, course_id),
                    FOREIGN KEY (student_id) REFERENCES students(id) ON DELETE CASCADE,
                    FOREIGN KEY (course_id) REFERENCES courses(id) ON DELETE CASCADE
                );
            """)
            cur.execute("""
                CREATE TABLE IF NOT EXISTS group_courses (
                    group_name TEXT NOT NULL,
                    course_id INTEGER NOT NULL,
                    PRIMARY KEY (group_name, course_id),
                    FOREIGN KEY (course_id) REFERENCES courses(id) ON DELETE CASCADE
                );
            """)
            conn.commit()
            print("OK: таблицы student_courses и group_courses готовы.")
    except sqlite3.Error as e:
        print(f"ensure_relations_tables error: {e}")

def add_course(course_name: str, instructor: str, credits: int) -> Optional[int]:
    try:
        with _get_conn() as conn:
            cur = conn.cursor()
            cur.execute(
                "INSERT INTO courses (course_name, instructor, credits) VALUES (?, ?, ?);",
                (course_name, instructor, credits)
            )
            return cur.lastrowid
    except sqlite3.IntegrityError as e:
        print(f"add_course integrity error: {e}")
        return None
    except sqlite3.Error as e:
        print(f"add_course error: {e}")
        return None

def enroll_student_in_course(student_id: int, course_id: int) -> bool:
    try:
        with _get_conn() as conn:
            cur = conn.cursor()
            cur.execute("SELECT 1 FROM students WHERE id = ?;", (student_id,))
            if cur.fetchone() is None:
                print("Нет такого студента.")
                return False
            cur.execute("SELECT 1 FROM courses WHERE id = ?;", (course_id,))
            if cur.fetchone() is None:
                print("Нет такого курса.")
                return False
            cur.execute("INSERT OR IGNORE INTO student_courses (student_id, course_id) VALUES (?, ?);", (student_id, course_id))
            return True
    except sqlite3.Error as e:
        print(f"enroll_student_in_course error: {e}")
        return False

def set_group_course(group_name: str, course_id: int) -> bool:
    try:
        with _get_conn() as conn:
            cur = conn.cursor()
            cur.execute("SELECT 1 FROM courses WHERE id = ?;", (course_id,))
            if cur.fetchone() is None:
                print("Нет такого курса.")
                return False
            cur.execute("INSERT OR IGNORE INTO group_courses (group_name, course_id) VALUES (?, ?);", (group_name, course_id))
            return True
    except sqlite3.Error as e:
        print(f"set_group_course error: {e}")
        return False

def get_student_courses(student_id: int) -> List[Dict[str, Any]]:
    try:
        with _get_conn() as conn:
            cur = conn.cursor()
            cur.execute("""
                SELECT c.*
                FROM courses c
                JOIN student_courses sc ON sc.course_id = c.id
                WHERE sc.student_id = ?
                ORDER BY c.id;
            """, (student_id,))
            return [dict(r) for r in cur.fetchall()]
    except sqlite3.Error as e:
        print(f"get_student_courses error: {e}")
        return []

def get_course_students(course_id: int) -> List[Dict[str, Any]]:
    try:
        with _get_conn() as conn:
            cur = conn.cursor()
            cur.execute("""
                SELECT s.*
                FROM students s
                JOIN student_courses sc ON sc.student_id = s.id
                WHERE sc.course_id = ?
                ORDER BY s.id;
            """, (course_id,))
            return [dict(r) for r in cur.fetchall()]
    except sqlite3.Error as e:
        print(f"get_course_students error: {e}")
        return []

def transfer_student(student_id: int, new_group_name: str) -> bool:
    try:
        conn = _get_conn()
        cur = conn.cursor()
        try:
            cur.execute("BEGIN;")

            cur.execute("SELECT group_name FROM students WHERE id = ?;", (student_id,))
            row = cur.fetchone()
            if row is None:
                raise ValueError("Студент не найден")
            old_group = row["group_name"]

            cur.execute("UPDATE students SET group_name = ? WHERE id = ?;", (new_group_name, student_id))

            cur.execute("DELETE FROM student_courses WHERE student_id = ?;", (student_id,))

            cur.execute("SELECT course_id FROM group_courses WHERE group_name = ?;", (new_group_name,))
            course_ids = [r["course_id"] for r in cur.fetchall()]
            cur.executemany(
                "INSERT OR IGNORE INTO student_courses (student_id, course_id) VALUES (?, ?);",
                [(student_id, cid) for cid in course_ids]
            )

            conn.commit()
            print(f"OK: студент {student_id} переведен {old_group} -> {new_group_name}, курсы обновлены.")
            return True
        except Exception as e:
            conn.rollback()
            print(f"transfer_student rollback: {e}")
            return False
        finally:
            conn.close()
    except sqlite3.Error as e:
        print(f"transfer_student db error: {e}")
        return False

ensure_relations_tables()

c1 = add_course("Математика", "Проф. Иванов", 5)
c2 = add_course("Физика", "Проф. Петрова", 4)
c3 = add_course("Программирование", "Доц. Сидоров", 6)

if c1: set_group_course("ГР-01", c1)
if c3: set_group_course("ГР-01", c3)
if c2: set_group_course("ГР-02", c2)
if c1: set_group_course("ГР-03", c1)
if c2: set_group_course("ГР-03", c2)

enroll_student_in_course(1, 1)
enroll_student_in_course(1, 3)
enroll_student_in_course(3, 2)

print("Курсы студента 1:", get_student_courses(1))
transfer_student(1, "ГР-03")
print("Курсы студента 1 после перевода:", get_student_courses(1))


OK: таблицы student_courses и group_courses готовы.
Курсы студента 1: [{'id': 1, 'course_name': 'Математика', 'instructor': 'Проф. Иванов', 'credits': 5, 'created_at': '2026-01-18 17:00:56'}, {'id': 3, 'course_name': 'Программирование', 'instructor': 'Доц. Сидоров', 'credits': 6, 'created_at': '2026-01-18 17:00:56'}]
OK: студент 1 переведен ГР-01 -> ГР-03, курсы обновлены.
Курсы студента 1 после перевода: [{'id': 1, 'course_name': 'Математика', 'instructor': 'Проф. Иванов', 'credits': 5, 'created_at': '2026-01-18 17:00:56'}, {'id': 2, 'course_name': 'Физика', 'instructor': 'Проф. Петрова', 'credits': 4, 'created_at': '2026-01-18 17:00:56'}]


In [4]:
# ЗАДАНИЕ 4
import sqlite3
from typing import Optional, Any, List, Dict

class UniversityDB:
    def __init__(self, db_path: str = "university.db"):
        self.db_path = db_path
        self.connection: Optional[sqlite3.Connection] = None
        self.cursor: Optional[sqlite3.Cursor] = None

    def __enter__(self):
        self.connection = sqlite3.connect(self.db_path)
        self.connection.execute("PRAGMA foreign_keys = ON;")
        self.connection.row_factory = sqlite3.Row
        self.cursor = self.connection.cursor()
        return self

    def __exit__(self, exc_type, exc_val, exc_tb):
        if self.connection:
            if exc_type is None:
                self.connection.commit()
            else:
                self.connection.rollback()
            self.connection.close()
        self.connection = None
        self.cursor = None
        return False

    def execute_query(self, query: str, params: Optional[tuple] = None) -> bool:
        try:
            if params is None:
                params = ()
            self.cursor.execute(query, params)
            return True
        except sqlite3.Error as e:
            print(f"execute_query error: {e}")
            return False

    def fetch_all(self, query: str, params: Optional[tuple] = None) -> List[Dict[str, Any]]:
        try:
            if params is None:
                params = ()
            self.cursor.execute(query, params)
            return [dict(r) for r in self.cursor.fetchall()]
        except sqlite3.Error as e:
            print(f"fetch_all error: {e}")
            return []

    def fetch_one(self, query: str, params: Optional[tuple] = None) -> Optional[Dict[str, Any]]:
        try:
            if params is None:
                params = ()
            self.cursor.execute(query, params)
            r = self.cursor.fetchone()
            return dict(r) if r else None
        except sqlite3.Error as e:
            print(f"fetch_one error: {e}")
            return None

    def get_student_statistics(self) -> Dict[str, Any]:
        total = self.fetch_one("SELECT COUNT(*) AS cnt FROM students;") or {"cnt": 0}
        avg = self.fetch_one("SELECT AVG(average_grade) AS avg_grade FROM students WHERE average_grade IS NOT NULL;") or {"avg_grade": None}
        by_groups = self.fetch_all("SELECT group_name, COUNT(*) AS cnt FROM students GROUP BY group_name ORDER BY group_name;")
        return {
            "total_students": total["cnt"],
            "university_avg_grade": avg["avg_grade"],
            "students_by_group": by_groups
        }

    def get_top_students(self, limit: int = 5) -> List[Dict[str, Any]]:
        return self.fetch_all("""
            SELECT id, first_name, last_name, group_name, admission_year, average_grade
            FROM students
            WHERE average_grade IS NOT NULL
            ORDER BY average_grade DESC, id ASC
            LIMIT ?;
        """, (limit,))

    def get_group_statistics(self) -> List[Dict[str, Any]]:
        return self.fetch_all("""
            SELECT
                group_name,
                COUNT(*) AS students_cnt,
                AVG(average_grade) AS avg_grade,
                MIN(average_grade) AS min_grade,
                MAX(average_grade) AS max_grade
            FROM students
            GROUP BY group_name
            ORDER BY group_name;
        """)

with UniversityDB("university.db") as db:
    print(db.get_student_statistics())
    print(db.get_top_students(3))
    print(db.get_group_statistics())


{'total_students': 2, 'university_avg_grade': 4.4, 'students_by_group': [{'group_name': 'ГР-02', 'cnt': 1}, {'group_name': 'ГР-03', 'cnt': 1}]}
[{'id': 1, 'first_name': 'Иван', 'last_name': 'Петров', 'group_name': 'ГР-03', 'admission_year': 2023, 'average_grade': 4.9}, {'id': 3, 'first_name': 'Алексей', 'last_name': 'Сидоров', 'group_name': 'ГР-02', 'admission_year': 2023, 'average_grade': 3.9}]
[{'group_name': 'ГР-02', 'students_cnt': 1, 'avg_grade': 3.9, 'min_grade': 3.9, 'max_grade': 3.9}, {'group_name': 'ГР-03', 'students_cnt': 1, 'avg_grade': 4.9, 'min_grade': 4.9, 'max_grade': 4.9}]


In [5]:
# ЗАДАНИЕ 5
import sqlite3
from typing import Optional, List, Dict, Any

def _print_table(rows: List[Dict[str, Any]], columns: List[str]) -> None:
    if not rows:
        print("(пусто)")
        return
    widths = {c: max(len(c), max(len(str(r.get(c, ""))) for r in rows)) for c in columns}
    header = " | ".join(c.ljust(widths[c]) for c in columns)
    sep = "-+-".join("-" * widths[c] for c in columns)
    print(header)
    print(sep)
    for r in rows:
        print(" | ".join(str(r.get(c, "")).ljust(widths[c]) for c in columns))

def _read_int(prompt: str, min_v: Optional[int]=None, max_v: Optional[int]=None) -> int:
    while True:
        try:
            v = int(input(prompt).strip())
            if min_v is not None and v < min_v:
                print(f"Число должно быть >= {min_v}")
                continue
            if max_v is not None and v > max_v:
                print(f"Число должно быть <= {max_v}")
                continue
            return v
        except ValueError:
            print("Введите целое число.")

def _read_float(prompt: str, min_v: Optional[float]=None, max_v: Optional[float]=None) -> float:
    while True:
        try:
            v = float(input(prompt).strip().replace(",", "."))
            if min_v is not None and v < min_v:
                print(f"Число должно быть >= {min_v}")
                continue
            if max_v is not None and v > max_v:
                print(f"Число должно быть <= {max_v}")
                continue
            return v
        except ValueError:
            print("Введите число.")

def add_student_interactive():
    print("\n--- Добавление студента ---")
    first_name = input("Имя: ").strip()
    last_name = input("Фамилия: ").strip()
    group_name = input("Группа: ").strip()
    admission_year = _read_int("Год поступления (2000-2030): ", 2000, 2030)
    grade_in = input("Средний балл (пусто если нет): ").strip()
    avg_grade = None
    if grade_in:
        try:
            avg_grade = float(grade_in.replace(",", "."))
        except ValueError:
            print("Некорректный балл, сохраню как NULL.")
            avg_grade = None

    with UniversityDB(DB_PATH) as db:
        ok = db.execute_query(
            "INSERT INTO students (first_name, last_name, group_name, admission_year, average_grade) VALUES (?, ?, ?, ?, ?);",
            (first_name, last_name, group_name, admission_year, avg_grade)
        )
        if ok:
            sid = db.fetch_one("SELECT last_insert_rowid() AS id;")["id"]
            print(f"OK: студент добавлен, id={sid}")
        else:
            print("Ошибка при добавлении.")

def display_all_students():
    with UniversityDB(DB_PATH) as db:
        rows = db.fetch_all("SELECT id, first_name, last_name, group_name, admission_year, average_grade FROM students ORDER BY id;")
    _print_table(rows, ["id", "first_name", "last_name", "group_name", "admission_year", "average_grade"])

def find_student_by_lastname():
    q = input("Введите фамилию (часть): ").strip()
    with UniversityDB(DB_PATH) as db:
        rows = db.fetch_all("""
            SELECT id, first_name, last_name, group_name, admission_year, average_grade
            FROM students
            WHERE last_name LIKE ?
            ORDER BY id;
        """, (f"%{q}%",))
    _print_table(rows, ["id", "first_name", "last_name", "group_name", "admission_year", "average_grade"])

def update_student_grade_interactive():
    sid = _read_int("ID студента: ", 1, None)
    new_grade = _read_float("Новый средний балл (0-5): ", 0.0, 5.0)
    with UniversityDB(DB_PATH) as db:
        ok = db.execute_query("UPDATE students SET average_grade = ? WHERE id = ?;", (new_grade, sid))
        changed = db.fetch_one("SELECT changes() AS c;")["c"] if ok else 0
    print("OK" if changed else "Не найден студент или не изменилось.")

def delete_student_interactive():
    sid = _read_int("ID студента: ", 1, None)
    with UniversityDB(DB_PATH) as db:
        ok = db.execute_query("DELETE FROM students WHERE id = ?;", (sid,))
        changed = db.fetch_one("SELECT changes() AS c;")["c"] if ok else 0
    print("OK" if changed else "Не найден студент.")

def show_statistics():
    with UniversityDB(DB_PATH) as db:
        st = db.get_student_statistics()
        top = db.get_top_students(5)
        grp = db.get_group_statistics()

    print("\n--- Статистика ---")
    print("Всего студентов:", st["total_students"])
    print("Средний балл по универу:", None if st["university_avg_grade"] is None else round(st["university_avg_grade"], 3))
    print("\nСтудентов по группам:")
    _print_table(st["students_by_group"], ["group_name", "cnt"])
    print("\nТОП студентов:")
    _print_table(top, ["id", "first_name", "last_name", "group_name", "average_grade"])
    print("\nСтатистика по группам:")
    _print_table(grp, ["group_name", "students_cnt", "avg_grade", "min_grade", "max_grade"])

def main():
    create_database()
    while True:
        print("\n=== Университетский учет ===")
        print("1. Добавить студента")
        print("2. Просмотреть всех студентов")
        print("3. Найти студента по фамилии")
        print("4. Обновить оценку студента")
        print("5. Удалить студента")
        print("6. Показать статистику")
        print("7. Выход")

        choice = input("Выберите действие: ").strip()

        if choice == "1":
            add_student_interactive()
        elif choice == "2":
            display_all_students()
        elif choice == "3":
            find_student_by_lastname()
        elif choice == "4":
            update_student_grade_interactive()
        elif choice == "5":
            delete_student_interactive()
        elif choice == "6":
            show_statistics()
        elif choice == "7":
            break
        else:
            print("Неверный выбор!")

main()

OK: university.db и таблицы students/courses созданы.

=== Университетский учет ===
1. Добавить студента
2. Просмотреть всех студентов
3. Найти студента по фамилии
4. Обновить оценку студента
5. Удалить студента
6. Показать статистику
7. Выход


Выберите действие:  2


id | first_name | last_name | group_name | admission_year | average_grade
---+------------+-----------+------------+----------------+--------------
1  | Иван       | Петров    | ГР-03      | 2023           | 4.9          
3  | Алексей    | Сидоров   | ГР-02      | 2023           | 3.9          

=== Университетский учет ===
1. Добавить студента
2. Просмотреть всех студентов
3. Найти студента по фамилии
4. Обновить оценку студента
5. Удалить студента
6. Показать статистику
7. Выход


Выберите действие:  7
